In [10]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt
import backtrader.indicators as btind

from backtrader.feeds import GenericCSVData

In [11]:
class dix_csv(GenericCSVData):
    
    lines= ('dix', 'gex')
    
    params = (
        ('dtformat', '%m/%d/%Y'),
        ('datetime', 0),
        ('time', -1),
        ('open', 1),
        ('high', 2),
        ('low', 3),
        ('close', 4),
        ('volume', 6),
        ('openinterest', -1),
        ('dix', 9),
        ('gex', 10),        
    )

In [13]:
#Optimize DIX MA
class DIXmaopt(bt.Strategy):
    params = (
        ('period', 2),
        ('printlog', False),
        ('dixupper', 46),
        ('dixlower', 40),
    )

    def log(self, txt, dt=None, doprint=False):
        ''' Logging function fot this strategy'''
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        #Add a DIXSMA Indicator
        self.dixsma = bt.indicators.MovingAverageSimple(
           self.data.dix, period=self.params.period)


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enough cash
        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # buy when dix greater than dixupper threshold
            if self.dixsma >= (self.params.dixupper/100):
                self.log('BUY CREATE, %.2f' % self.dataclose[0])
                self.order = self.buy()
                
        else:
            
            # sell when DIX less than dixlower threshold
            self.duration = len(self) - self.bar_executed + 1
            if self.dixsma <= (self.params.dixlower/100):
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()
    def stop(self):
        self.log('(DIXUpper %2d) (DIXLower%2d) (MA Period%2d) Ending Value %.2f' %
                 (self.params.dixupper, self.params.dixlower, self.params.period, self.broker.getvalue()), doprint=True)
        
#runDIXma Strategy Opt
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()    
    
    # Add a strategy
    strats = cerebro.optstrategy(
        DIXmaopt,
        period=range(1,10),
        dixupper=range(41,51),
        dixlower=range(34,43)) 
    
    
    # Datas are in a subfolder of the samples. Need to find where the script is
    # because it could have been called from anywhere
    modpath = os.path.dirname(os.path.abspath(sys.argv[0]))
    datapath = os.path.join(modpath, 'C:\\Users\\big_u\\DIX MA Testing Backtrader\DIX OHLC.csv')

    # Create a Data Feed
    data = dix_csv(
        dataname=datapath,
    )

    # Add the Data Feed to Cerebro
    cerebro.adddata(data)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Add a FixedSize sizer according to the stake
    cerebro.addsizer(bt.sizers.FixedSize, stake=1)

    # Print out the starting conditions
    #print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    cerebro.run(maxcpus=1)
    
    # Set the commission
    cerebro.broker.setcommission(commission=0.00)

    # Print out the final result
    #print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())


2020-07-02, (DIXUpper 41) (DIXLower34) (MA Period 1) Ending Value 101717.11
2020-07-02, (DIXUpper 41) (DIXLower35) (MA Period 1) Ending Value 101717.06
2020-07-02, (DIXUpper 41) (DIXLower36) (MA Period 1) Ending Value 101771.46
2020-07-02, (DIXUpper 41) (DIXLower37) (MA Period 1) Ending Value 101373.76
2020-07-02, (DIXUpper 41) (DIXLower38) (MA Period 1) Ending Value 101661.30
2020-07-02, (DIXUpper 41) (DIXLower39) (MA Period 1) Ending Value 102074.32
2020-07-02, (DIXUpper 41) (DIXLower40) (MA Period 1) Ending Value 101937.71
2020-07-02, (DIXUpper 41) (DIXLower41) (MA Period 1) Ending Value 101824.74
2020-07-02, (DIXUpper 41) (DIXLower42) (MA Period 1) Ending Value 101919.47
2020-07-02, (DIXUpper 42) (DIXLower34) (MA Period 1) Ending Value 101885.42
2020-07-02, (DIXUpper 42) (DIXLower35) (MA Period 1) Ending Value 101885.37
2020-07-02, (DIXUpper 42) (DIXLower36) (MA Period 1) Ending Value 101720.84
2020-07-02, (DIXUpper 42) (DIXLower37) (MA Period 1) Ending Value 101646.60
2020-07-02, 

2020-07-02, (DIXUpper 43) (DIXLower34) (MA Period 2) Ending Value 101911.52
2020-07-02, (DIXUpper 43) (DIXLower35) (MA Period 2) Ending Value 101805.62
2020-07-02, (DIXUpper 43) (DIXLower36) (MA Period 2) Ending Value 101656.48
2020-07-02, (DIXUpper 43) (DIXLower37) (MA Period 2) Ending Value 101432.12
2020-07-02, (DIXUpper 43) (DIXLower38) (MA Period 2) Ending Value 101127.80
2020-07-02, (DIXUpper 43) (DIXLower39) (MA Period 2) Ending Value 101440.76
2020-07-02, (DIXUpper 43) (DIXLower40) (MA Period 2) Ending Value 101654.23
2020-07-02, (DIXUpper 43) (DIXLower41) (MA Period 2) Ending Value 102019.03
2020-07-02, (DIXUpper 43) (DIXLower42) (MA Period 2) Ending Value 102269.38
2020-07-02, (DIXUpper 44) (DIXLower34) (MA Period 2) Ending Value 101714.25
2020-07-02, (DIXUpper 44) (DIXLower35) (MA Period 2) Ending Value 101689.44
2020-07-02, (DIXUpper 44) (DIXLower36) (MA Period 2) Ending Value 101485.72
2020-07-02, (DIXUpper 44) (DIXLower37) (MA Period 2) Ending Value 101440.18
2020-07-02, 

2020-07-02, (DIXUpper 45) (DIXLower34) (MA Period 3) Ending Value 101789.83
2020-07-02, (DIXUpper 45) (DIXLower35) (MA Period 3) Ending Value 101891.48
2020-07-02, (DIXUpper 45) (DIXLower36) (MA Period 3) Ending Value 101802.86
2020-07-02, (DIXUpper 45) (DIXLower37) (MA Period 3) Ending Value 101702.48
2020-07-02, (DIXUpper 45) (DIXLower38) (MA Period 3) Ending Value 101617.89
2020-07-02, (DIXUpper 45) (DIXLower39) (MA Period 3) Ending Value 102014.84
2020-07-02, (DIXUpper 45) (DIXLower40) (MA Period 3) Ending Value 101943.11
2020-07-02, (DIXUpper 45) (DIXLower41) (MA Period 3) Ending Value 101736.13
2020-07-02, (DIXUpper 45) (DIXLower42) (MA Period 3) Ending Value 101833.12
2020-07-02, (DIXUpper 46) (DIXLower34) (MA Period 3) Ending Value 101408.51
2020-07-02, (DIXUpper 46) (DIXLower35) (MA Period 3) Ending Value 101517.01
2020-07-02, (DIXUpper 46) (DIXLower36) (MA Period 3) Ending Value 101333.64
2020-07-02, (DIXUpper 46) (DIXLower37) (MA Period 3) Ending Value 101330.14
2020-07-02, 

2020-07-02, (DIXUpper 47) (DIXLower34) (MA Period 4) Ending Value 101952.10
2020-07-02, (DIXUpper 47) (DIXLower35) (MA Period 4) Ending Value 101465.03
2020-07-02, (DIXUpper 47) (DIXLower36) (MA Period 4) Ending Value 101459.73
2020-07-02, (DIXUpper 47) (DIXLower37) (MA Period 4) Ending Value 101455.24
2020-07-02, (DIXUpper 47) (DIXLower38) (MA Period 4) Ending Value 101435.33
2020-07-02, (DIXUpper 47) (DIXLower39) (MA Period 4) Ending Value 101385.38
2020-07-02, (DIXUpper 47) (DIXLower40) (MA Period 4) Ending Value 101358.85
2020-07-02, (DIXUpper 47) (DIXLower41) (MA Period 4) Ending Value 101354.78
2020-07-02, (DIXUpper 47) (DIXLower42) (MA Period 4) Ending Value 101313.64
2020-07-02, (DIXUpper 48) (DIXLower34) (MA Period 4) Ending Value 101920.25
2020-07-02, (DIXUpper 48) (DIXLower35) (MA Period 4) Ending Value 101523.07
2020-07-02, (DIXUpper 48) (DIXLower36) (MA Period 4) Ending Value 101517.77
2020-07-02, (DIXUpper 48) (DIXLower37) (MA Period 4) Ending Value 101513.28
2020-07-02, 

2020-07-02, (DIXUpper 49) (DIXLower34) (MA Period 5) Ending Value 100551.73
2020-07-02, (DIXUpper 49) (DIXLower35) (MA Period 5) Ending Value 100551.73
2020-07-02, (DIXUpper 49) (DIXLower36) (MA Period 5) Ending Value 100551.73
2020-07-02, (DIXUpper 49) (DIXLower37) (MA Period 5) Ending Value 100551.73
2020-07-02, (DIXUpper 49) (DIXLower38) (MA Period 5) Ending Value 100551.73
2020-07-02, (DIXUpper 49) (DIXLower39) (MA Period 5) Ending Value 100551.73
2020-07-02, (DIXUpper 49) (DIXLower40) (MA Period 5) Ending Value 100551.73
2020-07-02, (DIXUpper 49) (DIXLower41) (MA Period 5) Ending Value 100551.73
2020-07-02, (DIXUpper 49) (DIXLower42) (MA Period 5) Ending Value 100551.73
2020-07-02, (DIXUpper 50) (DIXLower34) (MA Period 5) Ending Value 100035.59
2020-07-02, (DIXUpper 50) (DIXLower35) (MA Period 5) Ending Value 100035.59
2020-07-02, (DIXUpper 50) (DIXLower36) (MA Period 5) Ending Value 100035.59
2020-07-02, (DIXUpper 50) (DIXLower37) (MA Period 5) Ending Value 100035.59
2020-07-02, 

2020-07-02, (DIXUpper 41) (DIXLower34) (MA Period 7) Ending Value 101829.75
2020-07-02, (DIXUpper 41) (DIXLower35) (MA Period 7) Ending Value 101750.22
2020-07-02, (DIXUpper 41) (DIXLower36) (MA Period 7) Ending Value 101759.31
2020-07-02, (DIXUpper 41) (DIXLower37) (MA Period 7) Ending Value 101776.16
2020-07-02, (DIXUpper 41) (DIXLower38) (MA Period 7) Ending Value 101521.03
2020-07-02, (DIXUpper 41) (DIXLower39) (MA Period 7) Ending Value 101554.15
2020-07-02, (DIXUpper 41) (DIXLower40) (MA Period 7) Ending Value 101453.99
2020-07-02, (DIXUpper 41) (DIXLower41) (MA Period 7) Ending Value 101381.92
2020-07-02, (DIXUpper 41) (DIXLower42) (MA Period 7) Ending Value 101611.85
2020-07-02, (DIXUpper 42) (DIXLower34) (MA Period 7) Ending Value 101849.80
2020-07-02, (DIXUpper 42) (DIXLower35) (MA Period 7) Ending Value 101729.46
2020-07-02, (DIXUpper 42) (DIXLower36) (MA Period 7) Ending Value 101508.27
2020-07-02, (DIXUpper 42) (DIXLower37) (MA Period 7) Ending Value 101504.46
2020-07-02, 

2020-07-02, (DIXUpper 43) (DIXLower34) (MA Period 8) Ending Value 101809.37
2020-07-02, (DIXUpper 43) (DIXLower35) (MA Period 8) Ending Value 101862.30
2020-07-02, (DIXUpper 43) (DIXLower36) (MA Period 8) Ending Value 101661.71
2020-07-02, (DIXUpper 43) (DIXLower37) (MA Period 8) Ending Value 101864.90
2020-07-02, (DIXUpper 43) (DIXLower38) (MA Period 8) Ending Value 101571.61
2020-07-02, (DIXUpper 43) (DIXLower39) (MA Period 8) Ending Value 101523.31
2020-07-02, (DIXUpper 43) (DIXLower40) (MA Period 8) Ending Value 102237.19
2020-07-02, (DIXUpper 43) (DIXLower41) (MA Period 8) Ending Value 101782.74
2020-07-02, (DIXUpper 43) (DIXLower42) (MA Period 8) Ending Value 102255.73
2020-07-02, (DIXUpper 44) (DIXLower34) (MA Period 8) Ending Value 101937.12
2020-07-02, (DIXUpper 44) (DIXLower35) (MA Period 8) Ending Value 101965.04
2020-07-02, (DIXUpper 44) (DIXLower36) (MA Period 8) Ending Value 101366.34
2020-07-02, (DIXUpper 44) (DIXLower37) (MA Period 8) Ending Value 101383.31
2020-07-02, 

2020-07-02, (DIXUpper 45) (DIXLower34) (MA Period 9) Ending Value 101989.54
2020-07-02, (DIXUpper 45) (DIXLower35) (MA Period 9) Ending Value 101989.54
2020-07-02, (DIXUpper 45) (DIXLower36) (MA Period 9) Ending Value 101570.67
2020-07-02, (DIXUpper 45) (DIXLower37) (MA Period 9) Ending Value 101582.48
2020-07-02, (DIXUpper 45) (DIXLower38) (MA Period 9) Ending Value 101453.88
2020-07-02, (DIXUpper 45) (DIXLower39) (MA Period 9) Ending Value 101517.12
2020-07-02, (DIXUpper 45) (DIXLower40) (MA Period 9) Ending Value 102003.37
2020-07-02, (DIXUpper 45) (DIXLower41) (MA Period 9) Ending Value 101909.51
2020-07-02, (DIXUpper 45) (DIXLower42) (MA Period 9) Ending Value 101946.12
2020-07-02, (DIXUpper 46) (DIXLower34) (MA Period 9) Ending Value 101953.32
2020-07-02, (DIXUpper 46) (DIXLower35) (MA Period 9) Ending Value 101953.32
2020-07-02, (DIXUpper 46) (DIXLower36) (MA Period 9) Ending Value 101523.90
2020-07-02, (DIXUpper 46) (DIXLower37) (MA Period 9) Ending Value 101535.71
2020-07-02, 